### INIZIALIZATION

#### define Node class

In [167]:
from collections import deque # used in the alghoritms

'''
Due to the unknown dataset, we cannot indetify different nodes with the same value, so we don't allow duplicate nodes
'''

class Node:
    def __init__(self, value=None):
        self.__depth: int = 0
        self.__children = []
        self.__value = value
        self.__parent: Node = None
    
    def add_parent(self, parent):
        self.__parent = parent

    def add_child(self, child):
        # we need this check due to duplicate nodes, in this way we only insert a duplicate child node one time
        if child.__depth == 0: 
            child.__depth = self.__depth + 1
            self.__children.append(child)
    
    def get_value(self):
       return self.__value
    
    def get_children(self):
       return self.__children
    
    def get_sequence(self):
        
        if self.__parent is None:
            return f"{self.__value} -> "
        
        path = self.__parent.get_sequence()
        return  path + f"{self.__value} -> "

    def get_depth(self) -> int:
        return self.__depth

       
    


def print_tree(node: Node, level=0, visited=None):
    if visited is None: # Lock infinite loop (cycle)
        visited = set()

    if node and node.get_value() not in visited:
        print("\t" * level + str(node.get_value()))
        visited.add(node.get_value())
        for child in node.get_children():
            print_tree(child, level + 1, visited)

#### make graph starting from a dataset

In [168]:
import sys

def make_graph(filename) -> Node:
    nodes = {}

    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue  # Ignore comment line

            parts = line.split()
            from_node = int(parts[0])
            to_node = int(parts[1])

            if from_node not in nodes:
                nodes[from_node] = Node(from_node)

            if to_node not in nodes:
                nodes[to_node] = Node(to_node)
                nodes[to_node].add_parent( nodes[from_node] )

            nodes[from_node].add_child(nodes[to_node])


    return nodes[0] # return only the first node


# graph = make_graph("dataset/Amazon0302_custom.txt")
graph = make_graph("dataset/Amazon0302.txt")
# graph = make_graph("dataset/p2p-Gnutella04.txt")
# graph = make_graph("dataset/p2p-Gnutella31.txt")

sys.setrecursionlimit(10000)
# print_tree(graph)

### UNINFORMED SEARCH

#### breadth first search

In [169]:
def breadth_first_search(first_node: Node, value_to_find):
    node_viewed = set()
    fringe = deque([first_node])  # using deque for efficient pop from the front

    while fringe:
        to_check_node = fringe.popleft() # use as FIFO queue
        node_viewed.add(to_check_node)

        if to_check_node.get_value() == value_to_find:
            print(f"Found node: {to_check_node.get_value()}")
            print(f"Sequence found: {to_check_node.get_sequence()[:-4]}")
            print(f"Nodes expanded: {len(node_viewed)}")
            return


        for child in __expand(to_check_node):
            if child not in node_viewed:
                fringe.append(child)

    print(f"Node {value_to_find} not found in the graph.")


def __expand(node: Node) -> list[Node]:
    return node.get_children()



breadth_first_search(graph, 189182)


Found node: 189182
Sequence found: 0 -> 4 -> 17 -> 31 -> 85 -> 170 -> 217 -> 330 -> 375 -> 401 -> 727 -> 1750 -> 2979 -> 4272 -> 6726 -> 17365 -> 21909 -> 22505 -> 22745 -> 38245 -> 41805 -> 72025 -> 115529 -> 171157 -> 189182
Nodes expanded: 91713


#### depth first search

In [170]:
def depth_first_search(first_node: Node, value_to_find):
    node_viewed = set()
    fringe = deque([first_node])

    while fringe:
        to_check_node = fringe.pop() # use as LIFO queue
        node_viewed.add(to_check_node)

        if to_check_node.get_value() == value_to_find:
            print(f"Found node: {to_check_node.get_value()}")
            print(f"Sequence found: {to_check_node.get_sequence()[:-4]}")
            print(f"Nodes expanded: {len(node_viewed)}")
            return


        for child in __expand(to_check_node):
            if child not in node_viewed:
                fringe.append(child)

    print(f"Node {value_to_find} not found in the graph.")


def __expand(node: Node) -> list[Node]:
    return reversed( node.get_children() ) # reversed to check first the left nodes



depth_first_search(graph, 189182)

Found node: 189182
Sequence found: 0 -> 4 -> 17 -> 31 -> 85 -> 170 -> 217 -> 330 -> 375 -> 401 -> 727 -> 1750 -> 2979 -> 4272 -> 6726 -> 17365 -> 21909 -> 22505 -> 22745 -> 38245 -> 41805 -> 72025 -> 115529 -> 171157 -> 189182
Nodes expanded: 88394


#### limited depth first search

In [171]:
def limited_depth_first_search(first_node: Node, value_to_find, max_depth: int):
    node_viewed = set()
    fringe = deque([first_node])

    while fringe:
        to_check_node = fringe.pop() # use as LIFO queue
        node_viewed.add(to_check_node)

        if to_check_node.get_value() == value_to_find:
            print(f"Found node: {to_check_node.get_value()}")
            print(f"Node's depth: {to_check_node.get_depth()}")
            print(f"Sequence found: {to_check_node.get_sequence()[:-4]}")
            print(f"Nodes expanded: {len(node_viewed)}")
            return

        for child in __expand(to_check_node):
            if child not in node_viewed and child.get_depth() <= max_depth:
                fringe.append(child)

    print(f"Node {value_to_find} not found in the graph.")
    print(f"Nodes expanded: {len(node_viewed)}")


def __expand(node: Node) -> list[Node]:
    return reversed( node.get_children() ) # reversed to check first the left nodes


print("search with l >= d:")
limited_depth_first_search(graph, 189182, 30)

print("\nsearch with l < d")
limited_depth_first_search(graph, 189182, 10)

search with l >= d:
Found node: 189182
Node's depth: 24
Sequence found: 0 -> 4 -> 17 -> 31 -> 85 -> 170 -> 217 -> 330 -> 375 -> 401 -> 727 -> 1750 -> 2979 -> 4272 -> 6726 -> 17365 -> 21909 -> 22505 -> 22745 -> 38245 -> 41805 -> 72025 -> 115529 -> 171157 -> 189182
Nodes expanded: 72295

search with l < d
Node 189182 not found in the graph.
Nodes expanded: 3286


#### iterative deepening search

In [172]:
# TODO: Implement the iterative deepening search



### CLEAN

In [173]:
del graph